In [ ]:
#Flow:
#setup
#Load data
#process data
#explore the data
#   preiew some random images
#create piepelines   
#use the pipelines
#user input
#
#
#
#

In [ ]:
###############################################################################################################################################################
#----------------------------------------------------------------------SETUP-----------------------------------------------------------------------------------
###############################################################################################################################################################

In [ ]:
###############################################################################################################################################################
#----------------------------------------------------------------------DATA PROCESSING-------------------------------------------------------------------------
###############################################################################################################################################################

In [ ]:
###############################################################################################################################################################
#----------------------------------------------------------------------EXPLORATORY ANALYSIS--------------------------------------------------------------------
###############################################################################################################################################################

In [ ]:
###############################################################################################################################################################
#----------------------------------------------------------------------CREATING THE PIPELINES------------------------------------------------------------------
###############################################################################################################################################################

In [ ]:
###############################################################################################################################################################
#----------------------------------------------------------------------APPLYING THE PIPELINES------------------------------------------------------------------
###############################################################################################################################################################

In [ ]:
###############################################################################################################################################################
#----------------------------------------------------------------------USER INPUT------------------------------------------------------------------------------
###############################################################################################################################################################